In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Data Understanding

In [7]:
# sheet name = 0 to choose the chines data
df = pd.read_excel('Données.xlsx',sheet_name=0 )

In [3]:
df.head()

,date,Indice des actions,Température,Emission CO2,Emission CO2.1,Chinese Yuan to US $,Taux d'intérêt,Prix du pétrole
0,date,PCH,TCH,CO2CH exprimés en Millions,CO2CH,CHUS,ICH,Oil
1,2019-01-01 00:00:00,2611.38,2.79,32.344424,32344423.8,6.8785,3.273,50.57
2,2019-01-02 00:00:00,2581.37,3.23,31.791466,31791466.3,6.862,3.203,54.06
3,2019-01-03 00:00:00,2580.43,6.2,30.711864,30711863.9,6.87205,3.191,53.23
4,2019-01-04 00:00:00,2633.35,8.87,31.512141,31512141,6.8692,3.198,55.64


In [8]:
# check the features types
df.dtypes

date                    object
Indice des actions      object
Température             object
Emission CO2            object
Emission CO2.1          object
Chinese Yuan to US $    object
Taux d'intérêt          object
Prix du pétrole         object
dtype: object

In [9]:
# check for null values
df.isnull().sum().sort_values(ascending = False)

date                    0
Indice des actions      0
Température             0
Emission CO2            0
Emission CO2.1          0
Chinese Yuan to US $    0
Taux d'intérêt          0
Prix du pétrole         0
dtype: int64

## Notes
* **drop the first row**
* **we need to change the types of this columuns `[Indice des actions, Température, Emission CO2, Chinese Yuan to US, Chinese Yuan to US $, Taux d'intérêt, Prix du pétrole]` to `float`**
* **choose to drop one feature between `Emission CO2` and `Emission CO2.1 ` to avoid repetition**